# Data Wrangling with Lytics Profile Data - Tools and Techniques

The goal of this notebook is to present some tools and techniques that can be used to wrangle Industry Dive data. 

## What is Data Wrangling again?
>Data wrangling, sometimes referred to as data munging, is the process of transforming and mapping data from one "raw" data form into another format with the intent of making it more appropriate and valuable for a variety of downstream purposes such as analytics.  Some transformation techniques include: parsing, joining, standardizing, augmenting, cleansing, and consolidating. 

[per wikipedia](https://en.wikipedia.org/wiki/Data_wrangling)

## Bad Data in, Bad Data out

![bad data in bad data out](https://cdn-images-1.medium.com/max/1200/0*YCghEemt6BtW9OZV.png "Bad Data in Bad Data out")

Many websites contain forms in order to collect information from users for various reasons.  In our case, we have signup forms for dives that asks for information about our users like so:

![signup form](../data/img/signup_form.png "signup form")

As you can see, there are fields that are restricted to pre-defined values (e.g., Job Function), and free-form fields (e.g., Company Name) where a user can type most anything they like.  Whenever users are exposed to free-form fields, there is a possibility of bad/messy/non-standardized data making into your system.

For example, here are some variants of "IKEA" that are present for user profiles that we have:

* IKEA
* IKEA AG
* IKEA Belgium
* IKEA Canada
* IKEA Danville
* IKEA Food
* IKEA Home Furnishings
* IKEA Portugal
* IKEA USA
* IKEA US EAST, LLC 215
* IKEA US

Without some wrangling, you would not be able to aggregate these folks properly into a single group based on company.

## Lytics Profile Data
Now, let's take a look at some Lytics profile data, which consists of all information we have about users who interact with our content.  Within this data, there are key demographic fields that can help us understand who our users are, such as:
* first and last name
* job title
* email domain
* company name
* address

The data file we are going to look at is an export of the "All" audience segment in Lytics.
https://activate.getlytics.com/audiences/4cc5d612f46fb86e5cfd0c995250e60c/summary?aid=2751

![All Audience segment in Lytics](../data/img/lytics_all_audience_segment.png "All Audience segment in Lytics")

Let's start looking at this data to see how we can clean it up in order to help us create more accurate statistics about our users.

In [39]:
import pandas as pd
import numpy as np

#dtypes = {'company': 'str', 'company_name': 'str', 'domain': 'object', 'emaildomain': 'object', 'emaildomains': 'object',
        # 'st_profile_id': 'object', 'user_id': np.float64, 'lytics_segment': 'object'}
df = pd.read_csv('../data/files/lytics_profile_data_export.csv', sep=',', error_bad_lines=False, index_col=False, encoding='latin-1')
                 #, dtype=dtypes)

# list columns in dataset
print(list(df))

# number of rows
print('# of rows left: %s' % df.shape[0])
# print(df[df['st_profile_id'].str.contains("5a2ba1f6ff530ac11a8b4868", na=False)])

['company_name', 'email', 'emaildomains', 'last_name', 'first_name', 'name', 'st_profile_id', 'city', 'state', 'country', 'zip', 'lytics_segment']
# of rows left: 684514


There are multiple fields in the data we can choose to cleanup, but first let's look at the "company_name" field.  One of the first things we should do is get rid of rows with company name values we don't care about.

In [40]:
# remove null company name values
df = df.dropna(subset=['company_name'])

# number of rows
print('# of rows left: %s' % df.shape[0])

# of rows left: 428204


In [41]:
# find values that are any combination of special characters
special_char_values = df['company_name'].str.contains("^[!@#$%^&*(),.?]*$", na=False)
print(df[special_char_values].company_name.unique())

# number of rows
print('# of special character value rows: %s' % df[special_char_values].shape[0])
df = df[~special_char_values]

print('# of rows left: %s' % df.shape[0])
# print(df[df['st_profile_id'].str.contains("5a2ba1f6ff530ac11a8b4868", na=False)])

['.' '********' '..' '...' '*' '.....' '......' ',' '????????' '??????'
 '?' ',,' '**' '@@' '???']
# of special character value rows: 97
# of rows left: 428107


In [42]:
# find values that are only numbers
number_values = df['company_name'].str.contains("^[0-9]*$", na=False)
print(df[number_values].company_name.unique())

# number of rows
print('# of number value rows: %s' % df[number_values].shape[0])
df = df[~number_values]

print('# of rows left: %s' % df.shape[0])

['1988' '252' '1963' '1957' '1965' '1997' '1954' '1949' '123' '111' '1970'
 '1968' '1990' '1979' '1975' '1974' '1989' '1967' '1984' '104' '1959'
 '1977' '1992' '0' '900' '1986' '1993' '605' '1998' '1969' '1985' '1972'
 '1000' '5' '1948' '1999' '1953' '451' '1995' '1960' '1980' '8020' '8760'
 '1987' '1956' '53' '1996' '1991' '1973' '1994' '1958' '2714' '1955'
 '1928' '1978' '1961' '1935' '1' '1947' '43' '15' '50' '1971' '34' '1946'
 '2016' '1952' '271' '1976' '1982' '1966' '1690' '47723' '1964' '2' '1940'
 '1983' '198' '2013' '1942' '5358359981' '151' '1951' '9172077326' '1950'
 '2008' '2020' '13' '295' '6' '411' '1962' '2015' '83255804' '359' '1945'
 '2009' '136' '438' '963' '32000' '309' '59' '1981' '8001504151' '14'
 '825' '404' '68' '525' '555' '789243438' '308' '1107' '673282495' '365'
 '3211' '2040' '6164381822' '40041466109' '10' '745' '1776' '12' '61'
 '2005' '3300909815' '8121066529' '130' '173' '3' '458' '2019' '777'
 '701010' '2435500285' '352158191' '28' '983' '347' '256' '1

In [43]:
# random additional values that I found when I was looking at the data in Excel
weird_vals = ['#NAME?', '{Re}', '< self >']
weird_values = df['company_name'].isin(weird_vals)
df = df[~weird_values]

# left over rows in dataframe
print('# of rows left: %s' % df.shape[0])

# of rows left: 427432


Now that we have cleaned all the bad company name values from our dataset, let's work on standardizing the names to help with comparison.

In [44]:
# change the values to all lower case
df['stndrdzed_company_name'] = df['company_name'].str.lower()
# remove all punctuation
df["stndrdzed_company_name"] = df['stndrdzed_company_name'].str.replace('[^\w\s]','')

# remove rows with "none" as value
none_rows = df['stndrdzed_company_name'].str.contains('none', na=False)
df = df[~none_rows]

# remove rows with "" as value
empty_string_rows = df['stndrdzed_company_name'].values == ''
df = df[~empty_string_rows]
print('# of rows left: %s' % df.shape[0])

# of rows left: 426539


Let's take a look at our dataset to see what we are working with:

In [45]:
grouped = df.groupby('stndrdzed_company_name')

grouped = grouped.size().reset_index(name='counts')
grouped.sort_values(by=['counts'], ascending=False)

,stndrdzed_company_name,counts
416,20160506deleteme,1302
202602,self,644
109066,ibm,510
248957,walmart,495
3477,accenture,483
138980,macys,423
217753,student,415
68501,duke energy,404
154431,mr,369
202620,self employed,365


One thing to note from looking at this is that there are company names that contain values other than English.  For instance, "현대엔지니어링" is Korean.  This is one thing you could work on eliminating as well if you wanted to focus on English values.  I tried to use a library called "langdetect" for this, but it did not do a good job of picking up the obvious cases.

Once we have wrangled the data bit, we can now try to enhance our dataset with an external dataset.  One of the datasets we bought rights to recently, DiscoverOrg, has different information about companies that could be useful for analysis.  The common field these two datasets have is the company name.  So we can try to load this dataset, clean it up a bit, then compare it to our original cleaned dataset in order to try and match on company name and enhance our existing dataset.

In [46]:
dtypes= {'Company ID': np.int64, 'Company Name': 'str', 'Company Website': 'object', 'Company HQ Phone': 'object',
        'Company Email Domain': 'object', 'Company Description': 'object', 'Company Primary Industry': 'object',
        'Company Revenue': np.float64, 'Company IT Budget (Mil)': 'object', 'Number of Employees': np.int64,
        'Company IT Employees': np.float64, 'Company Fortune Rank': np.float64, 'Company Ownership': 'object', 'Company Profile URL': 'object',
        'Company Business Model (B2B/B2C/B2G)': 'object', 'Hospital Beds': 'object', 'HQ Address 1': 'object', 'HQ Address 2': 'object',
        'HQ City': 'object', 'HQ State': 'object', 'HQ Postal Code': 'object', 'HQ County': 'object', 'HQ Country': 'object'
        }
df2 = pd.read_csv('../data/files/DiscoverOrg_Company_223030_20180731141156.csv', encoding='latin-1', sep=',', error_bad_lines=False, index_col=False, dtype=dtypes)

# change the values to all lower case
df2['stndrdzed_company_name'] = df2['Company Name'].astype(str).str.lower()
# remove all punctuation
df2["stndrdzed_company_name"] = df2['stndrdzed_company_name'].str.replace('[^\w\s]','')

In [55]:
# merge with discovery org data in order to find matches
merged_rows = pd.merge(df, df2, how= 'left', on= 'stndrdzed_company_name', sort=True, suffixes=('_a', '_b'),)

# number of rows after merging
print('# of rows left: %s' % merged_rows.shape[0])

# of rows left: 92940


So, after our intial cleaning process, we had 456,521 rows in our lytics file.  Our DiscoverOrg file had 68,735 rows.  By merging the two files on company name we were able to match 98,811 rows.  That is not a bad start.

Next, we will write the merged and non-merged rows to a file for further analysis.

In [58]:
import os
path=r'/Users/sasanbahadaran/Downloads'
merged_rows.to_csv(os.path.join(path,r'lytics_profile_disc_org_merged_rows.csv'), index=False)

Next, we can work on wrangling other fields and joining them to the DiscoveryOrg data for enhancement, such as:
* email domain
* address

We could also decide to work on further cleaning up the company data as well.  This could be through performing additional wrangling after examining our output file, or going beyond the deterministic types of methods we have covered so far.  All in all though, the more we standardize our dataset, the better results we will get when performing analysis on our data.

Ideas: Take address, standardize and clean, match up to discOrg and see what additional results you can yield.  Take email domain and do the same.  Filter out more junk based on analysis.